# Packages

In [98]:
!pip -q install langchain openai chromadb

# Conversation chain with memory

In [99]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.schema.messages import SystemMessage
from langchain.schema import HumanMessage
from langchain.chains import LLMChain,RetrievalQA
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import CharacterTextSplitter


In [100]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

# LLM
llm = ChatOpenAI()
system_prompt = """"you work as an assistant at a garment store. your goal is to ask the user about the
following:\n- the occasion they are shopping for\n- any particular type of garment they want\n- any preference of colors\n\n
Ask for all these preferences from the user one by one and combine the responses in the end in one sentence.
"""

# Prompt
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            system_prompt
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)

# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(llm=llm, prompt=prompt, verbose=True, memory=memory)

# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation({"question": "hi, i want to buy something, can you help me ?"})
conversation({"question": "i would like to get something for the beach"})
conversation({"question": "maybe a bikini"})
conversation({"question": "shades of blue i like"})
response = conversation({"question": "yes!"})
get_text = response['text']
get_text



> Entering new LLMChain chain...
Prompt after formatting:
System: "you work as an assistant at a garment store. your goal is to ask the user about the
following:
- the occasion they are shopping for
- any particular type of garment they want
- any preference of colors


Ask for all these preferences from the user one by one and combine the responses in the end in one sentence. Do not ask the user to confirm the summary.

Human: hi, i want to buy something, can you help me ?

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: "you work as an assistant at a garment store. your goal is to ask the user about the
following:
- the occasion they are shopping for
- any particular type of garment they want
- any preference of colors


Ask for all these preferences from the user one by one and combine the responses in the end in one sentence. Do not ask the user to confirm the summary.

Human: hi, i want to buy something, can you help me ?
AI: Of course! I'd 

"Great! I'll keep that in mind as I assist you in finding the perfect bikini for your beach outing. Let's get started!"

# Preparing DB

In [101]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [102]:
import pandas as pd
articles = pd.read_csv("/content/drive/MyDrive/Copy of articles.csv")
print(articles.columns)

def concat_columns(df, cols_to_concat, new_col_name, sep=","):
    df[new_col_name] = df[cols_to_concat[0]]
    for col in cols_to_concat[1:]:
        df[new_col_name] = df[new_col_name] + sep + df[col]
    return df

cols_to_concat = ['prod_name', 'product_type_name', 'product_group_name','graphical_appearance_name', 'colour_group_name', 'garment_group_name','detail_desc']
new_col_name = 'concatenated_desc'
articles = concat_columns(articles, cols_to_concat, new_col_name, sep=",")
# articles = articles[:10]
print(len(articles))
articles.head(n=2)

Index(['article_id', 'prod_name', 'product_type_name', 'product_group_name',
       'graphical_appearance_name', 'colour_group_name', 'garment_group_name',
       'detail_desc'],
      dtype='object')
1000


,article_id,prod_name,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,garment_group_name,detail_desc,concatenated_desc
0,695255001,Siv t-shirt,T-shirt,Garment Upper body,All over pattern,Dark Blue,Jersey Fancy,Short-sleeved top in soft viscose jersey with ...,"Siv t-shirt,T-shirt,Garment Upper body,All ove..."
1,821115007,RICHIE SKIRT,Skirt,Garment Lower body,Check,Pink,Skirts,"Short, pleated skirt in woven fabric with a hi...","RICHIE SKIRT,Skirt,Garment Lower body,Check,Pi..."


In [103]:
loader = DataFrameLoader(articles[['article_id', 'concatenated_desc']],page_content_column="concatenated_desc")
data=loader.load()
text_splitter = CharacterTextSplitter(chunk_size=250, chunk_overlap=10)
texts = text_splitter.split_documents(data)
print(len(texts))
texts[5]

1000


Document(page_content='KELLY SHIRT S.0,Shirt,Garment Upper body,Stripe,Blue,Blouses,Shirt in airy cotton with a collar, buttons down the front, long sleeves with buttoned cuffs, and a rounded hem.', metadata={'article_id': 697564030})

In [104]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'
import os

## here we are using OpenAI embeddings but in future we will swap out to local embeddings
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

## Make a retriever

In [107]:
retriever = vectordb.as_retriever(search_kwargs = {"k":2})
print(retriever.search_type)
print(retriever.search_kwargs)

similarity
{'k': 2}


## Make a chain

In [108]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [109]:
"""Function to convert AI's summary of user's request into a question"""
def AItoQuestion(input_txt):
    llm = ChatOpenAI()
    system_prompt_3 = """"the input statement will mention an object description.convert that description from a
    statement to a question that looks for the nearest match to the description. for example, if the user says:
    you want a new red top, then rephrase it as: is there anything that nearly matches a red top ?"""


    messages = [
        SystemMessage(
            content=system_prompt_3
        ),
        HumanMessage(
            content=input_txt
        ),
    ]
    op_txt = llm(messages)
    return op_txt

get_question = str(AItoQuestion(get_text))[9:]
get_question

'Thank you! I\'m looking for a new bikini for my beach outing. Is there anything that nearly matches my description?"'

In [110]:
llm_response = qa_chain(get_question) #passing the generated question into retrieval chain to get the best match
print("Best Match:")
op_txt = str(llm_response['source_documents'][0])[9:]
op_txt

Best Match:


"ent='FANTASTIC ELASTIC cheeky breif,Swimwear bottom,Swimwear,Colour blocking,Black,Swimwear,Fully lined bikini bottoms with narrow sides.' metadata={'article_id': 523468002}"

# Final Response

In [111]:
"""Function to convert matched output to conversational tone"""
def ResultsToConversation(input_txt):
    llm = ChatOpenAI()
    system_prompt_3 = """"the input statement will contain the description of an object followed by a metadata that contains an article id.
    convert the input into a human -like message. For example,  if the input is : 'fancy black pants. metadata = {'article_id' : 75657766} ' then rephrase it as :
    'here is what i found ! :Fancy black pants, check out article id : 75657766 """


    messages = [
        SystemMessage(content=system_prompt_3),
          HumanMessage(content=input_txt)
    ]
    op_txt = llm(messages)
    return op_txt

get_response = str(ResultsToConversation(op_txt))
get_response

"content='Here is what I found! A fantastic elastic cheeky brief swimwear bottom. It features color blocking in black and is fully lined. Check out the article ID: 523468002.'"